In [ ]:

import soundfile as sf
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from pynput import keyboard
from scipy.io.wavfile import write as wav_write

In [4]:
import numpy as np
import csv
import time
import sounddevice as sd
import threading
import scipy.io.wavfile as wav
from pynput import keyboard

# Parameters for sound recording
sample_rate = 44100  # Hz
duration = 10  # seconds
audio_data = np.zeros((duration * sample_rate, 1), dtype=np.int16)

# File paths
audio_file = 'keyboard_sound.wav'
log_file = 'key_log.csv'

# Initialize the keystroke log file
with open(log_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key', 'Action', 'Timestamp'])

# Global variable to track the start time of audio recording
start_time_audio = None
stop_recording = False  # Flag to stop both audio and keyboard listeners

# Recording function for audio
def record_audio():
    global audio_data, start_time_audio
    print("Recording audio...")

    # Set the start time of the recording
    start_time_audio = time.time()  # Capture the timestamp when recording starts

    # Record the audio and store it in audio_data
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()

    print("Audio recording finished")

    # Save the recorded audio to a WAV file
    wav.write(audio_file, sample_rate, audio_data)
    print(f"Audio saved to {audio_file}")

# Keystroke listener function
def on_press(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char  # Normal keys
    except AttributeError:
        key_str = str(key)  # Special keys like shift, ctrl, etc.

    # Log the key press with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Pressed', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Pressed at {timestamp:.6f} seconds")

def on_release(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char
    except AttributeError:
        key_str = str(key)

    # Log the key release with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Released', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Released at {timestamp:.6f} seconds")

    # Stop listener if 'Esc' key is pressed
    if key == keyboard.Key.esc:
        global stop_recording
        stop_recording = True  # Set flag to stop both recordings
        return False

# Start recording audio in a separate thread
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# Start the keyboard listener in the main thread to avoid blocking
def start_keyboard_listener():
    with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

# Run the keyboard listener in a separate thread
keyboard_thread = threading.Thread(target=start_keyboard_listener)
keyboard_thread.start()

# Wait for both threads to finish, while checking for stop condition
while not stop_recording:
    time.sleep(0.1)  # Prevent high CPU usage by sleeping briefly

# Once 'Esc' is pressed, both threads will finish
audio_thread.join()
keyboard_thread.join()
print("Recording process finished.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Recording audio...
Key y Pressed at 2.670438 seconds
Key y Released at 2.753566 seconds
Key u Pressed at 2.994618 seconds
Key u Released at 3.082331 seconds
Key i Pressed at 3.312487 seconds
Key i Released at 3.406326 seconds
Key o Pressed at 3.614205 seconds
Key o Released at 3.722144 seconds
Key p Pressed at 3.934731 seconds
Key p Released at 4.051237 seconds
Key r Pressed at 4.688597 seconds
Key r Released at 4.759930 seconds
Key t Pressed at 4.956676 seconds
Key t Released at 5.045028 seconds
Key r Pressed at 5.248967 seconds
Key r Released at 5.329797 seconds
Key Key.esc Pressed at 6.187999 seconds
Key Key.esc Released at 6.247269 seconds
Audio recording finished
Audio saved to keyboard_sound.wav
Recording process finished.


In [6]:
# Load the audio file
audio_path = "keyboard_sound.wav"
audio_data, sample_rate = sf.read(audio_path)

# Read keystroke timestamps from CSV
keystroke_times = []
with open("key_log.csv", "r") as audio_data_file:
    reader = csv.reader(audio_data_file)
    stack = {}

    for row in reader:
        if row[0] == "Key":
            continue
        key = row[0]
        action = row[1]
        timestamp = float(row[2])

        if action == "Pressed":
            if key in stack:
                raise Exception(f"Key {key} was pressed again before being released.")
            stack[key] = [key, timestamp]

        elif action == "Released":
            if key not in stack:
                raise Exception(f"Key {key} was released without being pressed.")
            stack[key].append(timestamp)
            keystroke_times.append(stack[key])
            del stack[key]

print(keystroke_times)


NameError: name 'sf' is not defined

## The code that generates individual spectrograms for each keystroke

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import csv
import os

# Parameters
AUDIO_FILE = "keyboard_sound.wav"  # Replace with your actual audio file
KEYSTROKE_CSV = "key_log.csv"
OUTPUT_DIR = "keystroke_spectrograms/keystroke_spectrograms"
BUFFER_TIME = 0.2  # Extra time (seconds) before & after each keystroke

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load audio data
sample_rate, audio_data = wav.read(AUDIO_FILE)

# Read keystroke data from CSV
keystroke_times = []
with open(KEYSTROKE_CSV, "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    stack = {}

    for row in reader:
        key, action, timestamp = row[0], row[1], float(row[2])

        if action == "Pressed":
            stack[key] = timestamp  # Store press time
        elif action == "Released" and key in stack:
            keystroke_times.append((key, stack.pop(key), timestamp))  # Store key, press, and release

# Function to create and save the spectrogram
def create_spectrogram(audio_segment, key, idx):
    # Generate the spectrogram using scipy
    f, t, Sxx = signal.spectrogram(audio_segment, sample_rate)

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='auto', cmap='inferno')
    plt.colorbar(label='Power (dB)')
    plt.title(f"Keystroke '{key}' Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

    # Save the spectrogram to file
    spectrogram_path = os.path.join(OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.png")
    plt.savefig(spectrogram_path)
    plt.close()
    print(f"Saved spectrogram for '{key}' at {spectrogram_path}")

# Process keystrokes
for idx, (key, press_time, release_time) in enumerate(keystroke_times):
    # Define start and end times (add buffer)
    start_time = max(0, press_time - BUFFER_TIME)
    end_time = min(len(audio_data) / sample_rate, release_time + BUFFER_TIME)

    # Convert times to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)

    # Extract audio segment
    keystroke_audio = audio_data[start_sample:end_sample]

    if len(keystroke_audio) == 0:
        print(f"Warning: Empty audio segment for keystroke {idx + 1}")
        continue

    # Create and save the spectrogram for this key
    create_spectrogram(keystroke_audio, key, idx)

print("Processing complete. Spectrograms saved.")


Saved spectrogram for 'y' at keystroke_spectrograms/keystroke_spectrograms/keystroke_1_y.png
Saved spectrogram for 'u' at keystroke_spectrograms/keystroke_spectrograms/keystroke_2_u.png
Saved spectrogram for 'i' at keystroke_spectrograms/keystroke_spectrograms/keystroke_3_i.png
Saved spectrogram for 'o' at keystroke_spectrograms/keystroke_spectrograms/keystroke_4_o.png
Saved spectrogram for 'p' at keystroke_spectrograms/keystroke_spectrograms/keystroke_5_p.png
Saved spectrogram for 'r' at keystroke_spectrograms/keystroke_spectrograms/keystroke_6_r.png
Saved spectrogram for 't' at keystroke_spectrograms/keystroke_spectrograms/keystroke_7_t.png
Saved spectrogram for 'r' at keystroke_spectrograms/keystroke_spectrograms/keystroke_8_r.png
Saved spectrogram for 'Key.esc' at keystroke_spectrograms/keystroke_spectrograms/keystroke_9_Key.esc.png
Processing complete. Spectrograms saved.
